In [1]:
from functions import *
# import wordninja
import requests
import random
import pandas as pd
import os
import torch
import torch.nn as nn

# Torch cannot work properly in jupyter notebook
# import os
# count = 0 
# if count == 0:
#     os.chdir("test_dir")
#     count += 1


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
2024-04-10 13:08:13.460059: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-10 13:08:14.627862: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning

In [2]:
os.getcwd()

'/root/DSA4266_Grp2'

In [3]:
# pd.read_pickle(DF_PATH)

In [4]:
## CONFIG

DF_PATH = "Data/full_df_2.pkl"
X_NAME = 'clean_msg'
Y_NAME = 'class'
EMBEDDINGS_FOLDER = 'embeddings_2'

#### For preprocessing
MAXLEN_PER_SENT = 150
ALL_TOKEN_MAX_WORDS = 5000
INPUT_LENGTH = 150
UNDERSAMPLE = True
EPOCHS = 3
BATCH_SIZE = 125
NUM_CLASSES = 2
HIDDEN_SIZE = 75
LEARNING_RATE = 0.001
VERBOSE = True
NUM_LAYERS = 1

use_cuda = torch.cuda.is_available()
DEVICE = torch.device("cuda" if use_cuda else "cpu")
print(DEVICE)
torch.manual_seed(13)
torch.set_default_device(DEVICE)


cuda


In [5]:
## Semantic Dictionaries

def get_synonyms_conceptnet(word):
    synonyms = []
    url = f'http://api.conceptnet.io/c/en/{word}?filter=/c/en'
    response = requests.get(url)
    data = response.json()
    for edge in data['edges']:
        if edge['rel']['label'] == 'Synonym' and edge['start']['language'] == 'en' and edge['end']['language'] == 'en':
            start = edge['start']['label']
            end = edge['end']['label']
            synonyms.append(end if start == word else start)

    if synonyms != []:
        synonym = random.choice(synonyms)
    else:
        synonym = synonyms
    return synonym

def get_synonyms_wordnet(word):
    synonyms = []
    synsets = wordnet.synsets(word)
    for synset in synsets:
        synonyms.extend([lemma.name() for lemma in synset.lemmas() if lemma.name() != word])

    if synonyms != []:
        synonym = random.choice(synonyms)
    else:
        synonym = synonyms
    return synonym

In [6]:
class DataPrep():
    def __init__(self, subset = None, text_prep = 'lem', token_max_words = ALL_TOKEN_MAX_WORDS, maxlen_per_sent = MAXLEN_PER_SENT, undersample = UNDERSAMPLE):
        """
        subset: X[:subset]
        """
        self.df = pd.read_pickle(DF_PATH)
        self.subset = subset
        self.maxlen_per_sent = maxlen_per_sent


        self.remove_duplicates()
        print('Dupes removed')
        self.X = self.df[X_NAME]
        self.y = self.df[Y_NAME].apply(lambda x: 1 if x == 'spam' else 0)
        self.token_max_words = token_max_words

        if self.subset:
            self.X = self.X[:self.subset]
            self.y = self.y[:self.subset]
        
        print('Tokenizing..')
        self.tokenize()
        print('Finished Tokenizing')

        print('Initialising word2vec')
        self.word_to_vec_map = self.word2vec()

        print('lemm/stemm')
        if text_prep == 'lem':
            self.X = self.lemming()
        if text_prep == 'stem':
            self.X = self.stemming()

        print('Embedding...')
        self.pre_embed()
        path = f'{EMBEDDINGS_FOLDER}/emb_matrix_x{self.subset}_tok_{self.maxlen_per_sent}_len{self.token_max_words}.pkl'
        if os.path.exists(path):
            self.emb_matrix = pd.read_pickle(path)
        else:
            self.emb_matrix = self.tok_embedding_mat(alternative = [get_synonyms_conceptnet, get_synonyms_wordnet])
            print('Finished embedding')

        print('Padding')
        X_pad = self.pad()
        print('Finished padding')

        self.X_train, self.X_test, self.y_train, self.y_test = train_test_split(X_pad, self.y, test_size=0.20, random_state=42)
        self.X_train, self.X_val, self.y_train, self.y_val = train_test_split(self.X_train, self.y_train, test_size = 0.2, random_state=42 )

        if undersample:
            print('Undersampling..')
            print(Counter(self.y_train))
            self.X_train, self.y_train = self.undersample()
            print(Counter(self.y_train))




    def remove_duplicates(self):
    
        ## First remove all those X values with differing binary y values
        occurrences = self.df.groupby([X_NAME, Y_NAME]).size().reset_index(name='count')
        duplicates = occurrences[occurrences.duplicated(subset=X_NAME, keep=False)]
        for index, row in duplicates.iterrows():
            x_value = row[X_NAME]
            max_count = occurrences[(occurrences[X_NAME] == x_value)].max()['count']
            occurrences.drop(occurrences[(occurrences[X_NAME] == x_value) & (occurrences['count'] != max_count)].index, inplace=True)

        ## Remove duplicates
        self.df = occurrences.drop_duplicates(subset = X_NAME).reset_index(drop = True)
    
    def tokenize(self, join = False):
        def tokenize_helper(text, join = False):
            stop_words = set(stopwords.words('english'))
            tokens = word_tokenize(text)
            tokens = [word.lower() for word in tokens if word.lower() not in stop_words]

            if join:
                tokens = ' '.join([''.join(c for c in word if c not in string.punctuation) for word in tokens if word])
        
            return tokens
        
        self.X = self.X.apply(lambda x: tokenize_helper(x, join))

    ## Embedders
        
    def word2vec(self):
        from gensim.models.word2vec import Word2Vec
        import gensim.downloader as api

        word_to_vec_map = api.load("word2vec-google-news-300")

        return word_to_vec_map
    
    
    ## Stemming/ Lemmetization

    def stemming(self):
        ps = PorterStemmer()

        def stem(row):
            print(row)
            stemmed = []
            for word in row:
                stemmed += [ps.stem(word)]
            print('STEMMED:', stemmed)

            return stemmed

        return self.X.apply(stem)
    

    def lemming(self):

        def lem(row):
            lemmatizer = WordNetLemmatizer()
            lemmed = [lemmatizer.lemmatize(word) for word in row]
            # print(row)
            # print(lemmed,"\n")
            return lemmed

        return self.X.apply(lem)
    
    def pre_embed(self):
        self.tokenizer = text.Tokenizer(num_words=self.token_max_words)
        self.tokenizer.fit_on_texts(self.X)

        self.sequences = self.tokenizer.texts_to_sequences(self.X)

        self.word_index = self.tokenizer.word_index
        self.vocab_len = len(self.word_index) + 1
        self.embed_vector_len = self.word_to_vec_map['moon'].shape[0]
    
    def tok_embedding_mat(self, alternative):
        """
        embedder: word2vec
        alternative: list of callable to find synonyms from, inorder of precedence
        """
        synonyms = {} #Dict to store synonyms

        emb_matrix = np.zeros((self.vocab_len, self.embed_vector_len))


        for word, index in tqdm.tqdm(self.word_index.items(), total = len(self.word_index)):
            try: # Try to find in word2vec
                embedding_vector = self.word_to_vec_map[word]
                emb_matrix[index-1, :] = embedding_vector
            except: # Word2vec dont have, find in own synonym dict
                synonym = synonyms.get(word, None) 
                if (synonym) and (synonym in self.word_to_vec_map.index_to_key):
                    emb_matrix[index-1,:] = self.word_to_vec_map[synonym]
                else: # If word2vec, own synonym dict dont have, find from dictionaries
                    for dictionary in alternative:
                        try: 
                            synonym = dictionary(word)
                            if synonym:
                                # print(f'Found synonym: {synonym} for word: {word}')
                                embedding_vector = self.word_to_vec_map[synonym] 
                                emb_matrix[index-1, :] = embedding_vector
                                synonyms[word] = synonym
                        except:
                            continue
        self.syn = synonyms
        
        try:
            pd.to_pickle(emb_matrix, f"{EMBEDDINGS_FOLDER}/emb_matrix_x{self.subset}_tok_{self.maxlen_per_sent}_len{self.token_max_words}.pkl")
        except:
            print('Saved unsuccessfully')
            return emb_matrix

        return emb_matrix


    def pad(self):
        X_pad = pad_sequences(self.sequences, maxlen = self.maxlen_per_sent)
        return X_pad

    def undersample(self):
        undersampler = RandomUnderSampler(random_state=42)
        X_resampled, y_resampled = undersampler.fit_resample(self.X_train, self.y_train)

        return X_resampled, y_resampled


class Train(DataPrep):
    def __init__(self, subset = None, text_prep = 'lem', token_max_words = 5000, maxlen_per_sent = 150, undersample = True):
        super().__init__(subset, text_prep, token_max_words, maxlen_per_sent, undersample)
        
        use_cuda = torch.cuda.is_available()
        self.device = torch.device("cuda" if use_cuda else "cpu")

        self.X_train_tensor = torch.as_tensor(self.X_train, dtype = torch.float)
        self.y_train_tensor = torch.as_tensor(self.y_train, dtype = torch.int8)


    def lstm(self, nodes):

        self.model = Sequential().to(device = self.device)
        self.model.add(Embedding(input_dim= self.vocab_len, output_dim= self.embed_vector_len, input_shape = (self.maxlen_per_sent,), trainable=False, embeddings_initializer = initializers.Constant(self.emb_matrix)))
        self.model.add(LSTM(nodes))
        self.model.add(Dense(1, activation = 'sigmoid'))

        self.model.compile(optimizer='adam',
                    loss='binary_crossentropy',
                    metrics=['accuracy'])

        # Train model
        self.model.fit(self.X_train, self.y_train, epochs=10, batch_size=1, verbose=1)  
    
    # def lstm_op(self):
    #     import math

    #     def objective(trial):
    #         units = trial.suggest_categorical("units", [32, 64, 128, 256])
    #         units2 = units//2
    #         epochs = trial.suggest_categorical("epochs", [10, 20, 30])
    #         batch_size = trial.suggest_categorical("batch_size", [32, 64, 128])
    #         dropout = trial.suggest_float("dropout", low = 0.1, high = 0.5)
            
    #         self.model = Sequential()
    #         self.model.add(Embedding(input_dim= self.vocab_len, output_dim= self.embed_vector_len, input_shape = (self.maxlen_per_sent,), trainable=False, embeddings_initializer = initializers.Constant(self.emb_matrix)))
    #         self.model.add(LSTM(units))
    #         self.model.add(Dropout(dropout))
    #         self.model.add(Dense(units2))
    #         self.model.add(Dense(1, activation = 'sigmoid'))

    #         self.model.compile(optimizer='adam',
    #                         loss='binary_crossentropy',
    #                         metrics=['accuracy'])

    #         self.model.fit(self.X_train, self.y_train, epochs= epochs, batch_size= batch_size, verbose=1)  
    #         _, accuracy = self.model.evaluate(self.X_test, self.y_test, verbose=0)

    #         return accuracy

    #     study = optuna.create_study(direction="maximize")
    #     study.optimize(objective, n_trials=10)

    #     self.best_trial = study.best_trial
    #     self.best_params = self.best_trial.params
    #     self.best_accuracy = self.best_trial.value

    #     print("Best hyperparameters:", self.best_params)
    #     print("Best accuracy:", self.best_accuracy)


    def predict(self, verbose = False):

        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        print("Test Accuracy:", accuracy)

        # Make predictions
        predictions = self.model.predict(self.X_test)

        y_hat = [1 if i> 0.5 else 0 for i in predictions]

        if verbose:
            print("Classification Report:")
            print(classification_report(self.y_test, y_hat))

            print("Confusion Matrix:")
            print(confusion_matrix(self.y_test, y_hat))

    


In [7]:
test = Train()
## the test.X_train is not embedded


Dupes removed
Tokenizing..
Finished Tokenizing
Initialising word2vec
lemm/stemm
Embedding...
Padding
Finished padding
Undersampling..
Counter({0: 19480, 1: 5672})
Counter({0: 5672, 1: 5672})


In [8]:
test.X_train

array([[   0,    0,    0, ..., 1501, 1126,   25],
       [   0,    0,    0, ...,  205, 2790, 4265],
       [   0,    0,    0, ...,  274,  581, 4183],
       ...,
       [   0,    0,    0, ...,  867,   73,   28],
       [   0,    0,    0, ..., 1565, 1977, 2873],
       [   0,    0,    0, ...,   24,   73,   39]], dtype=int32)

In [9]:
class TokenEmbedding(nn.Module):
    def __init__(self, vocab_size, embed_size):
        super(TokenEmbedding, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_size)

    def forward(self, x):
        embedded = self.embedding(x)
        return embedded
        

embedding_layer = TokenEmbedding(vocab_size = ALL_TOKEN_MAX_WORDS, embed_size = MAXLEN_PER_SENT)
input_tensor_train = torch.as_tensor(test.X_train, dtype=torch.int64)
input_tensor_test = torch.as_tensor(test.X_test, dtype = torch.int64)
input_tensor_val = torch.as_tensor(test.X_val, dtype = torch.int64)

X_train = embedding_layer(input_tensor_train)
X_test = embedding_layer(input_tensor_test)
X_val = embedding_layer(input_tensor_val)

print("Input tensor shape:", input_tensor_train.shape)
print("Embedded output shape:", X_train.shape)
X_train

Input tensor shape: torch.Size([11344, 150])
Embedded output shape: torch.Size([11344, 150, 150])


tensor([[[ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         [ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         [ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         ...,
         [ 0.3236, -0.4857, -0.6464,  ...,  0.8669,  1.8450, -0.7716],
         [ 1.6670,  1.1885, -0.0464,  ...,  2.1215,  0.0046, -0.5499],
         [ 0.6553, -1.5342,  0.5770,  ...,  0.2437, -1.3944,  0.1840]],

        [[ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         [ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         [ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         ...,
         [-2.3411, -1.0062, -0.8563,  ..., -0.9615,  1.3050, -0.1689],
         [-1.3110,  0.6002,  1.7048,  ..., -0.2773, -2.0583,  1.4835],
         [-0.4248, -0.2866,  1.7164,  ...,  1.1965, -0.3353, -1.8403]],

        [[ 1.0607,  0.3815,  0.3934,  ..., -0.2935,  0.4256, -0.1617],
         [ 1.0607,  0.3815,  0.3934,  ..., -0

In [59]:
import torch
import torch.utils.data as Data

y_train_tensor = torch.as_tensor(test.y_train, dtype=torch.float32)
y_test_tensor = torch.as_tensor(test.y_test.to_numpy(dtype=np.single))
y_val_tensor = torch.as_tensor(test.y_val.to_numpy(dtype=np.single))

train_data = Data.TensorDataset(X_train, y_train_tensor)
test_data = Data.TensorDataset(X_test, y_test_tensor)
val_data = Data.TensorDataset(X_val, y_val_tensor)

train_loader = Data.DataLoader(dataset=train_data,
                               batch_size =BATCH_SIZE,
                               shuffle=False)

test_loader = Data.DataLoader(dataset=test_data,
                              batch_size=BATCH_SIZE,
                              shuffle=False)

val_loader = Data.DataLoader(dataset = val_data,
                            batch_size = BATCH_SIZE,
                            shuffle = False)

y_train_tensor.shape

torch.Size([11344])

In [99]:
import torch.nn.functional as F

class LSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, output_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()
        # self.hidden = nn.Linear(input_size, hidden_size)

    def forward(self, X):
        ##  (num layers, batch_size, hidden_size)
        hidden_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size, device = DEVICE)
        cell_states = torch.zeros(self.num_layers, X.size(0), self.hidden_size, device = DEVICE)

        out, _ = self.lstm(X, (hidden_states, cell_states))
        out = self.output_layer(out[:, -1, :])
        return self.sigmoid(out)

    def init_hidden(self, batch_size, device='cpu'):
        # Initializes hidden state
        # The hidden state is a tuple of (h_0, c_0) for LSTMs
        # h_0: Initial hidden state for each element in the batch
        # c_0: Initial cell state for each element in the batch
        h_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        c_0 = torch.zeros(self.num_layers, batch_size, self.hidden_size).to(device)
        return (h_0, c_0)


In [100]:
lstm = LSTMModel(input_size = INPUT_LENGTH, hidden_size = HIDDEN_SIZE, num_layers = NUM_LAYERS, output_size = 1).to(DEVICE)
print(lstm)
loss_func = nn.BCELoss()
optimizer = torch.optim.Adam(lstm.parameters(), lr = LEARNING_RATE)

LSTMModel(
  (lstm): LSTM(150, 75, batch_first=True)
  (output_layer): Linear(in_features=75, out_features=1, bias=True)
  (sigmoid): Sigmoid()
)


In [101]:
def accuracy_fn(y_true,y_pred):
    correct = torch.eq(y_true,y_pred).sum().item()
    accuracy = (correct / len(y_pred)) * 100
    return accuracy

def precision_fn(y_true,y_pred):
    true_positive = ((y_pred == 1.0) & (y_true == 1.0)).sum().item()
    predicted_positive = (y_pred==1.0).sum().item()
    return true_positive/(predicted_positive + 1e-7)

def recall_fn(y_true,y_pred):
    true_positive = ((y_pred == 1.0) & (y_true == 1.0)).sum().item()
    actual_positive = (y_true==1.0).sum().item()
    return true_positive/(actual_positive + 1e-7)

def f1_score(y_true,y_pred):
    prec = precision_fn(y_true,y_pred)
    recall = recall_fn(y_true,y_pred)
    return 2 * (prec * recall) / (prec + recall + 1e-7)

In [102]:
def loss_batch(model,loss_fn,xb,yb, opt=None):
    yb_pred = model(xb)

    print(yb_pred)
    print(yb)
    loss = loss_fn(yb_pred.squeeze(),yb)

    yb_bin = (yb_pred.squeeze() > 0.5).float()
    
    accuracy = accuracy_fn(yb,yb_bin)
    precision = precision_fn(yb,yb_bin)
    recall = recall_fn(yb,yb_bin)
    f1 = f1_score(yb,yb_bin)

    if opt is not None:
        opt.zero_grad()
        loss.backward(retain_graph = True)
        opt.step()
        

    return loss.item(), accuracy, precision, recall, f1, len(xb)
    

def train(num_epochs, model, train_dataloader, val_dataloader, opt = None):
    total_steps = len(train_dataloader)

    for epoch in range(num_epochs):
        for batch, (X_train_batch, y_train_batch) in tqdm.tqdm(enumerate(train_dataloader), total = len(train_loader), desc = f"Epochs {epoch+1}/{num_epochs}"):
            X_train_batch = X_train_batch.to(DEVICE)
            y_train_batch = y_train_batch.to(DEVICE)
            loss_batch(model, loss_func, X_train_batch, y_train_batch, opt)

        with torch.no_grad():
            losses, accuracy, precision,recall, f1_scores, nums = zip(*[loss_batch(model,loss_func,xb,yb) for xb,yb in val_dataloader]
            )

        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        val_accuracy = np.sum(np.multiply(accuracy,nums)) / np.sum(nums)
        val_precision = np.sum(np.multiply(precision,nums)) / np.sum(nums)
        val_recall = np.sum(np.multiply(recall,nums)) / np.sum(nums)
        val_f1_score = np.sum(np.multiply(f1_scores,nums)) / np.sum(nums)

        if (epoch+1) % 100 == 0:
            print(f"Epoch: {epoch+1}") 
            print(f"Validation Loss: {val_loss}")
            print(f"Validation Accuracy: {val_accuracy:.4f}% | Validation Recall: {val_recall:.4f} | Validation Precision: {val_precision:.4f} | Validation F1 Score: {val_f1_score:.4f}")   




In [103]:
trained_model = train(EPOCHS, lstm, train_loader, val_loader, optimizer)

Epochs 1/3:   0%|          | 0/91 [00:00<?, ?it/s]


tensor([[0.5441],
        [0.5123],
        [0.4943],
        [0.5128],
        [0.5107],
        [0.5149],
        [0.5201],
        [0.4992],
        [0.5033],
        [0.4830],
        [0.5183],
        [0.5581],
        [0.4937],
        [0.4859],
        [0.4836],
        [0.5183],
        [0.5423],
        [0.5240],
        [0.5112],
        [0.5039],
        [0.5361],
        [0.4965],
        [0.5195],
        [0.5046],
        [0.5115],
        [0.4635],
        [0.5199],
        [0.5519],
        [0.5049],
        [0.5153],
        [0.5302],
        [0.5460],
        [0.5163],
        [0.4964],
        [0.4900],
        [0.5200],
        [0.5401],
        [0.5309],
        [0.5102],
        [0.5215],
        [0.5496],
        [0.4909],
        [0.5013],
        [0.5179],
        [0.4925],
        [0.4888],
        [0.4879],
        [0.5463],
        [0.5549],
        [0.4819],
        [0.4946],
        [0.5331],
        [0.4783],
        [0.5102],
        [0.5354],
        [0

RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.

In [ ]:
torch.save(trained_model, 'model/model_epoch_10_bs_8.pth')

In [ ]:
model = torch.load('model/trained_model.pth')

In [ ]:
# _results = []
# for epoch in range(EPOCHS):
#     lstm.train()

#     train_loss = 0
#     for batch, (X_train_batch, y_train_batch) in enumerate(train_loader):

#         lstm.zero_grad()
#         out, prob = lstm(X_train_batch)

#         print(out)

#         loss = F.binary_cross_entropy(out, y_train_batch)

#         loss.backward()
#         optimizer.step()
#         train_loss += loss.item() * X_train_batch.size(0)

#     train_loss /= len(train_loader.dataset)

#     if VERBOSE:
#         print('Epoch: {}, Train Loss: {:4f}', format(epoch, train_loss))

#     lstm.eval()

#     with torch.no_grad():
#         correct = 0; valid_loss = 0
#         for i, (X_test_batch, y_test_batch) in enumerate(test_loader):
#             out, prob = lstm(X_test_batch)
#             loss = F.binary_cross_entropy(out, y_test_batch)

#             valid_loss += loss.item() * X_test_batch.size(0)

#             preds = prob.argmax(dim = 1, keepdim  = True)

#             correct += preds.eq(y_test_batch.view_as(preds)).sum().item() #Count number of correct

#         valid_loss /= len(test_loader.dataset)
#         accuracy = correct / len(test_loader.dataset)

#     if VERBOSE:
#         print('Validation Loss: {}, Validation Accuracy: {:4f}', format(valid_loss, accuracy))

#     _results.append([epoch, train_loss, valid_loss, accuracy])

# results = np.array(_results)
# print('Fin Train')
# print('FInal validation error: ', 100.(1-accuracy), "%")


In [21]:
test.lstm()

LSTMModel(
  (lstm): LSTM(34256, 512, batch_first=True)
  (fc): Linear(in_features=512, out_features=1, bias=True)
)

In [ ]:
test.train_lstm()

In [24]:
import torch
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print("Device: ",device)

Device:  cuda


In [ ]:
test.lstm_op()